In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
import tkinter as tk
from tkinter import messagebox

# -------------------------------
# 1) قراءة البيانات
# -------------------------------
df = pd.read_csv("train_Data.csv")
df_test = pd.read_csv("test_Data.csv")

# تنظيف البيانات
if "Unnamed: 0" in df.columns:
    df.drop(columns=["Unnamed: 0"], inplace=True)
if "Unnamed: 0" in df_test.columns:
    df_test.drop(columns=["Unnamed: 0"], inplace=True)

df = pd.get_dummies(df, drop_first=True)
df_test = pd.get_dummies(df_test, drop_first=True)

df.dropna(inplace=True)
df_test.dropna(inplace=True)

# -------------------------------
# 2) فصل الـ Features والـ Target
# -------------------------------
X = df.drop("diabetes", axis=1)
y = df["diabetes"]

# تقسيم Train/Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# -------------------------------
# 3) تدريب Random Forest
# -------------------------------
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=8,
    random_state=42
)
model.fit(X_train, y_train)

# تقييم مبدئي
y_pred = model.predict(X_test)
print("F1 Score:", f1_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# -------------------------------
# 4) GUI باستخدام Tkinter
# -------------------------------
def predict():
    try:
        # قراءة القيم من الإدخالات
        inputs = []
        for entry in entries:
            val = float(entry.get())
            inputs.append(val)

        # تحويل الإدخالات لـ DataFrame بنفس الأعمدة
        sample = pd.DataFrame([inputs], columns=labels)

        # عمل Scaling
        sample = scaler.transform(sample)

        # عمل التوقع
        result = model.predict(sample)[0]

        # عرض النتيجة
        if result == 1:
            messagebox.showinfo("Prediction", "Patient is Diabetic")
        else:
            messagebox.showinfo("Prediction", "Patient is NOT Diabetic")

    except ValueError:
        messagebox.showerror("Error", "Please enter valid numbers.")

# GUI window
root = tk.Tk()
root.title("Diabetes Prediction - Random Forest")

labels = list(X.columns)  # أسماء الأعمدة
entries = []

for i, col in enumerate(labels):
    tk.Label(root, text=col).grid(row=i, column=0, padx=10, pady=5, sticky="w")
    e = tk.Entry(root)
    e.grid(row=i, column=1, padx=10, pady=5)
    entries.append(e)

tk.Button(root, text="Predict", command=predict, bg="lightblue").grid(
    row=len(labels), column=0, columnspan=2, pady=20
)

root.mainloop()

F1 Score: 0.8042049934296978

Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98     14635
           1       1.00      0.67      0.80      1365

    accuracy                           0.97     16000
   macro avg       0.99      0.84      0.89     16000
weighted avg       0.97      0.97      0.97     16000

